In [1]:
import numpy as np
import pandas as pd
import os
import json

from datetime import datetime

with open('../config.json', 'r') as config_file:
    config = json.load(config_file)
imp_folder = config['raw_path']

In [ ]:
# Import the time-series csv files
csv_files = [file for file in os.listdir(imp_folder) if file.endswith('.csv')]
dataframes = {file: pd.read_csv(os.path.join(imp_folder, file)) for file in csv_files}

print(dataframes.keys())

dict_keys(['research_data2.csv', 'research_data.csv'])


In [3]:
print(dataframes['research_data.csv'].shape)
dataframes['research_data.csv'].head(10)

(31532117, 5)


,timestamp,name,MinRSSImain[dBm],PageSessionTotal,ULvolMByte
0,2023-09-01 00:30:00+02,"BtsId=997,CellId=0,CarrierId=0",-109.08,0,0.000000
1,2023-09-01 00:30:00+02,"BtsId=997,CellId=0,CarrierId=0",-109.08,0,0.000000
2,2023-09-01 00:30:00+02,"BtsId=997,CellId=0,CarrierId=1",-109.27,0,0.000907
3,2023-09-01 00:30:00+02,"BtsId=997,CellId=0,CarrierId=1",-109.27,0,0.000907
4,2023-09-01 00:30:00+02,"BtsId=187,CellId=0,CarrierId=0",-104.87,156,1.685871
5,2023-09-01 00:30:00+02,"BtsId=187,CellId=0,CarrierId=0",-104.87,0,1.685871
6,2023-09-01 00:30:00+02,"BtsId=187,CellId=0,CarrierId=1",-104.75,0,1.441159
7,2023-09-01 00:30:00+02,"BtsId=187,CellId=0,CarrierId=1",-104.75,141,1.441159
8,2023-09-01 00:30:00+02,"BtsId=187,CellId=1,CarrierId=0",-101.90,39,0.816284
9,2023-09-01 00:30:00+02,"BtsId=187,CellId=1,CarrierId=0",-101.90,0,0.816284


In [4]:
print(dataframes['research_data2.csv'].shape)
dataframes['research_data2.csv'].head(10)

(15946252, 5)


,timestamp,name,AvgSessionSetupDuration[s],AvgSessionDuration[s],TotalBlocks
0,2023-09-01 00:30:00+02,"BtsId=997,CellId=0,CarrierId=0",0.000000,0.000000,0
1,2023-09-01 00:30:00+02,"BtsId=997,CellId=0,CarrierId=1",6.000000,6.500000,0
2,2023-09-01 00:30:00+02,"BtsId=187,CellId=0,CarrierId=0",108.571429,115.428571,0
3,2023-09-01 00:30:00+02,"BtsId=187,CellId=0,CarrierId=1",50.844828,53.931034,0
4,2023-09-01 00:30:00+02,"BtsId=187,CellId=1,CarrierId=0",208.000000,218.500000,0
5,2023-09-01 00:30:00+02,"BtsId=187,CellId=1,CarrierId=1",411.500000,432.750000,0
6,2023-09-01 00:30:00+02,"BtsId=187,CellId=2,CarrierId=0",79.933333,85.166667,0
7,2023-09-01 00:30:00+02,"BtsId=187,CellId=2,CarrierId=1",100.807692,106.961538,0
8,2023-09-01 00:30:00+02,"BtsId=192,CellId=0,CarrierId=0",14.687500,15.859375,0
9,2023-09-01 00:30:00+02,"BtsId=192,CellId=0,CarrierId=1",11.405405,12.202703,0


In [5]:
# Step 1: Remove duplicates in research_data
# Keep rows where 'PageSessionTotal' is non-zero in case of duplicates
dataframes['research_data.csv'] = (
    dataframes['research_data.csv'].sort_values(by='PageSessionTotal', ascending=False)
    .drop_duplicates(subset=['timestamp', 'name'], keep='first')
)

# Step 2: Merge the cleaned dataframe with research_data2
df = pd.merge(dataframes['research_data.csv'], dataframes['research_data2.csv'], on=['timestamp', 'name'], how='outer').sort_values(by=['timestamp', 'name'])

# Column processing

KPIs

In [6]:
df.rename(columns={
    'MinRSSImain[dBm]': 'minRSSI',
    'PageSessionTotal': 'pageSessions',
    'ULvolMByte': 'ULvol',
    'AvgSessionSetupDuration[s]': 'sessionSetupDur',
    'AvgSessionDuration[s]': 'sessionDur',
    'TotalBlocks': 'blocks'
}, inplace=True)

In [7]:
kpis = [col for col in df.columns if col not in ['timestamp', 'name']]

Timestamps

In [8]:
df.head()

,timestamp,name,minRSSI,pageSessions,ULvol,sessionSetupDur,sessionDur,blocks
5247698,2023-09-01 00:30:00+02,"BtsId=154,CellId=0,CarrierId=1",-102.50,78.0,0.829355,17.457447,18.521277,0
3015680,2023-09-01 00:30:00+02,"BtsId=154,CellId=1,CarrierId=1",-103.84,159.0,1.450913,27.500000,29.200000,0
6045588,2023-09-01 00:30:00+02,"BtsId=154,CellId=2,CarrierId=1",-104.92,60.0,0.746502,4.558011,4.795580,0
6974462,2023-09-01 00:30:00+02,"BtsId=162,CellId=0,CarrierId=0",-104.19,43.0,0.436677,84.500000,88.785714,0
6695893,2023-09-01 00:30:00+02,"BtsId=162,CellId=0,CarrierId=1",-103.93,48.0,0.519392,90.785714,96.428571,0


In [9]:
def convert_to_naive(timestamp):
    dt_aware = datetime.fromisoformat(timestamp)         # Convert to timezone-aware datetime
    adjusted_time = dt_aware + dt_aware.utcoffset()      # Adjust for the UTC offset
    return adjusted_time.replace(tzinfo=None)            # Remove timezone info

df['timestamp'] = df['timestamp'].astype(str)
df['timestamp'] = df['timestamp'].apply(convert_to_naive)

df.head()

,timestamp,name,minRSSI,pageSessions,ULvol,sessionSetupDur,sessionDur,blocks
5247698,2023-09-01 02:30:00,"BtsId=154,CellId=0,CarrierId=1",-102.50,78.0,0.829355,17.457447,18.521277,0
3015680,2023-09-01 02:30:00,"BtsId=154,CellId=1,CarrierId=1",-103.84,159.0,1.450913,27.500000,29.200000,0
6045588,2023-09-01 02:30:00,"BtsId=154,CellId=2,CarrierId=1",-104.92,60.0,0.746502,4.558011,4.795580,0
6974462,2023-09-01 02:30:00,"BtsId=162,CellId=0,CarrierId=0",-104.19,43.0,0.436677,84.500000,88.785714,0
6695893,2023-09-01 02:30:00,"BtsId=162,CellId=0,CarrierId=1",-103.93,48.0,0.519392,90.785714,96.428571,0


Cells

In [ ]:
# Split the 'name' column into separate parts
df[['bts', 'antenna', 'carrier']] = df['name'].str.extract(r'BtsId=(\d+),CellId=(\d+),CarrierId=(\d+)')

# Create the new 'cell' column
df['cell'] = df['bts'] + '_' + df['antenna'] + '_' + df['carrier']

df = df.drop('name', axis=1)

In [11]:
columns_order = ['timestamp', 'cell', 'bts', 'antenna', 'carrier'] + kpis
df = df[columns_order]
df.head()

,timestamp,cell,bts,antenna,carrier,minRSSI,pageSessions,ULvol,sessionSetupDur,sessionDur,blocks
5247698,2023-09-01 02:30:00,154_0_1,154,0,1,-102.50,78.0,0.829355,17.457447,18.521277,0
3015680,2023-09-01 02:30:00,154_1_1,154,1,1,-103.84,159.0,1.450913,27.500000,29.200000,0
6045588,2023-09-01 02:30:00,154_2_1,154,2,1,-104.92,60.0,0.746502,4.558011,4.795580,0
6974462,2023-09-01 02:30:00,162_0_0,162,0,0,-104.19,43.0,0.436677,84.500000,88.785714,0
6695893,2023-09-01 02:30:00,162_0_1,162,0,1,-103.93,48.0,0.519392,90.785714,96.428571,0


# Missing values

In [12]:
df.isnull().sum()

timestamp               0
cell                    0
bts                     0
antenna                 0
carrier                 0
minRSSI            149133
pageSessions       149133
ULvol              149133
sessionSetupDur         0
sessionDur              0
blocks                  0
dtype: int64

In [13]:
print(dataframes['research_data.csv']['name'].nunique())
print(dataframes['research_data2.csv']['name'].nunique())

918
938


In [14]:
df = df.dropna(subset=['minRSSI'])

# Display the first few rows of the DataFrame to confirm
print(df.shape)

(15797119, 11)


In [15]:
unique_timestamps_per_cell = df.groupby('cell')['timestamp'].nunique()

min_value = unique_timestamps_per_cell.min()
max_value = unique_timestamps_per_cell.max()
mean_value = unique_timestamps_per_cell.mean()

print(f"Min: {min_value}, Max: {max_value}, Mean: {mean_value}")

Min: 3340, Max: 19086, Mean: 17207.200435729847


In [16]:
total_unique_timestamps = df['timestamp'].nunique()
total_unique_timestamps

19088

In [ ]:
threshold = mean_value

# Filter out 'name' entries that have less than the threshold number of unique timestamps
names_to_keep = unique_timestamps_per_cell[unique_timestamps_per_cell >= threshold].index
df = df[df['cell'].isin(names_to_keep)]

df.head(), print(df.shape)

(12926033, 11)


(                  timestamp     cell  bts antenna carrier  minRSSI  \
 5247698 2023-09-01 02:30:00  154_0_1  154       0       1  -102.50   
 3015680 2023-09-01 02:30:00  154_1_1  154       1       1  -103.84   
 6045588 2023-09-01 02:30:00  154_2_1  154       2       1  -104.92   
 6974462 2023-09-01 02:30:00  162_0_0  162       0       0  -104.19   
 6695893 2023-09-01 02:30:00  162_0_1  162       0       1  -103.93   
 
          pageSessions     ULvol  sessionSetupDur  sessionDur  blocks  
 5247698          78.0  0.829355        17.457447   18.521277       0  
 3015680         159.0  1.450913        27.500000   29.200000       0  
 6045588          60.0  0.746502         4.558011    4.795580       0  
 6974462          43.0  0.436677        84.500000   88.785714       0  
 6695893          48.0  0.519392        90.785714   96.428571       0  ,
 None)

Need to drop bts 995 to 999 (internal requirement).

In [ ]:
bts_to_drop = ['995', '996', '997', '998', '999']
df = df[~df['bts'].isin(bts_to_drop)]

print(df.shape)

(12741835, 11)


# Export

In [20]:
exp_folder = config['data_path']
df.to_csv(os.path.join(exp_folder, 'full_kpis.csv'), index=False)